<a href="https://colab.research.google.com/github/maytlim/doh_data/blob/main/vacancies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -s -O https://raw.githubusercontent.com/maytlim/doh_data/main/landline.py

In [2]:
import requests
import pandas as pd
import datetime
import landline
import gdown

In [3]:
  pd.options.display.float_format = '{:,.0f}'.format

In [4]:
try:
  import pdfx
except ModuleNotFoundError:
  !pip -q install pdfx
  import pdfx

     |████████████████████████████████| 178 kB 9.5 MB/s 
     |████████████████████████████████| 5.6 MB 46.8 MB/s 
     |████████████████████████████████| 3.0 MB 48.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires chardet<4,>=3.0.2, but you have chardet 4.0.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [5]:
!pip -q install xlsxwriter

     |████████████████████████████████| 148 kB 6.8 MB/s 


In [6]:
#Code from https://gist.github.com/korakot/51a917e1f53891d53be223439b0f75c1
from google.colab import auth
auth.authenticate_user()  # must authenticate

'''list all ids of files directly under folder folder_id'''
def folder_list(folder_id):
  from googleapiclient.discovery import build
  gdrive = build('drive', 'v3', cache_discovery=False).files()
  #gdrive = build('drive', 'v3').files()
  res = gdrive.list(q="'%s' in parents" % folder_id).execute()
  return [f['id'] for f in res['files']]

'''download all files from a gdrive folder to current directory'''
def folder_download(folder_id):
  for fid in folder_list(folder_id):
    !gdown -q --id $fid

In [7]:
ddrop_readme = requests.get('http://bit.ly/DataDropPH').url
ddrop_readme = ddrop_readme.split('/')[-1].split('?')[0]
#print(ddrop_readme)
folder_download(ddrop_readme)

In [8]:
readme_pdf = !ls *.pdf
readme_pdf = readme_pdf[0][1:-1]
readme_pdf
pdfx_tmp = pdfx.PDFx(readme_pdf)
ddlinks = pdfx_tmp.get_references_as_dict()

In [9]:
month_day = readme_pdf.split('_')
month = int(month_day[0][-2:])
day = int(month_day[1][:2])
month, day
year = 2021
print('DOH DataDrop Release:', year, month, day)

DOH DataDrop Release: 2021 8 30


In [10]:
downloaded = False
for ddlink in ddlinks['url']:
  if downloaded == False:
    if ddlink[:15] == 'https://bit.ly/' and ddlink[-3:] != 'ive':
      datadrop = requests.get(ddlink).url
      datadrop = datadrop.split('/')[-1].split('?')[0]
      folder_download(datadrop)
      print('Downloading DOH Data Drop...')
      downloaded = True
  else:
    continue

Permission denied: https://drive.google.com/uc?id=1Sk56niKz9tu7M0LRBIZX0BP-PXp7ro8ubAY44s5XE6U
Maybe you need to change permission over 'Anyone with the link'?


In [11]:
data_date = datetime.datetime(year, month, day)
#data_dir ='DOH COVID Data Drop_'  + data_date.strftime('%Y%m%d') + '/'

In [12]:
landline_masterlist = landline.make_masterlist()

In [13]:
# hospital daily patient census
cap_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 05 DOH Data Collect - Daily Report.csv'
# hospital weekly inventory of supplies
sup_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 06 DOH Data Collect - Weekly Report.csv'
print('weekly supplies inventory:', sup_filename)
print('daily capability count:', cap_filename)
hosp_code = 'hfhudcode' 
#hosp_name = 'cfname'

weekly supplies inventory: DOH COVID Data Drop_ 20210830 - 06 DOH Data Collect - Weekly Report.csv
daily capability count: DOH COVID Data Drop_ 20210830 - 05 DOH Data Collect - Daily Report.csv


In [14]:
#load capacity data
df_cap = pd.read_csv(cap_filename)
df_cap['cap_reportdate'] = pd.to_datetime(df_cap['reportdate'])
df_cap.drop(['updateddate', 'addeddate', 'reportdate', 'id', 'north_coord', 'east_coord'], axis=1, inplace=True)
df_cap[hosp_code] = df_cap[hosp_code].apply(lambda x: x.strip())

In [15]:
!gdown -q 'https://raw.githubusercontent.com/maytlim/doh_data/main/nhfr.json'

In [16]:
df_nhfr = pd.read_json('nhfr.json')
print('Refer to https://nhfr.doh.gov.ph/ for the latest data')

Refer to https://nhfr.doh.gov.ph/ for the latest data


In [17]:
df_cap = df_cap.merge(how='left', right=df_nhfr, left_on='hfhudcode', right_on='Health Facility Code')

In [18]:
#load supplies data
#df_sup = pd.read_csv(sup_filename)
#df_sup['sup_reportdate'] = pd.to_datetime(df_sup['reportdate'])
#df_sup.drop(['updateddate', 'addeddate', 'reportdate', 'id'], axis=1, inplace=True)
#df_sup[hosp_code] = df_sup[hosp_code].apply(lambda x: x.strip())

In [19]:
# quarantine facility daily patient census
#quarantine_daily = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 08 Quarantine Facility Data - Daily Report.csv')
# quarantine facility weekly inventory of supplies
#quarantine_weekly = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 09 Quarantine Facility Data - Weekly Report.csv')

In [20]:
# HCW availability
#baseline_v3 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 10 DOH Data Collect v3 - Baseline.csv')
# mechvent, swabbing, ipc
#baseline_v4 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 11 DOH Data Collect v4 - Baseline.csv')
# quarantine and isolation facilities
#ttmf = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 12 DDC TTMF Tracker v1 - Baseline.csv')

In [21]:
#print('{0}\n{1}'.format(df_cap.keys(), df_sup.keys()))
#print('{0}\n{1}'.format(quarantine_weekly.keys(), quarantine_daily.keys()))

In [22]:
reg_lookup = {'NCR': 'NATIONAL CAPITAL REGION (NCR)', 'ARMM': 'AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM)', 
              'CAR': 'CORDILLERA ADMINISTRA TIVE REGION (CAR)', 'I': 'REGION I (ILOCOS REGION)', 
              'II': 'REGION II (CAGAYAN VALLEY)', 'III': 'REGION III (CENTRAL LUZON)', 
              'IV-A': 'REGION IV-A (CALABAR ZON)', 'IV-B': 'REGION IV-B (MIMAROPA)', 
              'V': 'REGION V (BICOL REGION)', 'VI': 'REGION VI (WESTERN VISAYAS)', 
              'VII': 'REGION VII (CENTRAL VISAYAS)', 'VIII': 'REGION VIII (EASTERN VISAYAS)', 
              'IX': 'REGION IX (ZAMBOANGA PENINSULA)', 'X': 'REGION X (NORTHERN MINDANAO)', 
              'XI': 'REGION XI (DAVAO REGION)', 'XII': 'REGION XII (SOCCSKSA RGEN)', 
              'XIII': 'REGION XIII (CARAGA)'}

In [23]:
#prefix = ['ics', 'resdoc', 'nurse', 'medtech', 'respthe', 'radtech', 'support']
#suffix = ['_total', '_er', '_icu', '_ward'] 
#triage = ['resdoc_triage', 'nurse_triage', 'support_triage']
#added = ['Augmen_total']

In [24]:
def day_ago_from(n, from_date):
  showdate = from_date - datetime.timedelta(hours=24)*n
  return showdate.strftime('%Y-%m-%d')

In [25]:
def show_filtered(df_cap, days_ago, region='NATIONAL CAPITAL REGION (NCR)'):
  filter = (df_cap.mechvent_v > 0) & \
    (df_cap.cap_reportdate == day_ago_from(days_ago, data_date)) & \
    (df_cap.region == region) & \
    (df_cap.icu_v > 0) & \
    (df_cap.icu_o > 0)
  df_vacant = df_cap[filter][['cap_reportdate', 'cfname', 'city_mun', 'icu_v', 'isolbed_v', 'mechvent_v', 'beds_ward_v', 
                            'icu_o', 'isolbed_o', 'mechvent_o', 'beds_ward_o', 
                            'Landline Number', 'Landline Number 2', 'Fax Number', 'Ownership Major Classification']]
  df_vacant.rename({'Landline Number': 'landline', 'Landline Number 2': 'landline2', 'Fax Number': 'fax', 'Ownership Major Classification':'ownership'}, axis=1, inplace=True)
  df_vacant['landline'] = df_vacant['landline'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['landline2'] = df_vacant['landline2'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['fax'] = df_vacant['fax'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant.sort_values(by='icu_v', ascending=False, inplace=True)
  df_vacant.reset_index(inplace=True)
  return df_vacant

In [26]:
show_filtered(df_cap, days_ago=3)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,64208,2021-08-27,JOSE R. REYES MEMORIAL MEDICAL CENTER,SANTA CRUZ,79,49,13,0,12,73,6,45,(02) 8711 9491,(02) 8711 9498,(02) 8732 1077,Government
1,10567,2021-08-27,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,78,0,32,72,62,0,31,308,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
2,501531,2021-08-27,EAST AVENUE MEDICAL CENTER,QUEZON CITY,38,0,20,0,108,229,20,0,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
3,39957,2021-08-27,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,22,40,2,0,26,64,2,0,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
4,382615,2021-08-27,MAKATI MEDICAL CENTER,CITY OF MAKATI,12,0,10,25,17,87,10,40,(02) 8888 8999,(02) 8840 0737,(02) 8892 5593,Private
5,461876,2021-08-27,RIZAL MEDICAL CENTER,CITY OF PASIG,12,0,9,25,18,0,11,78,(02) 8865 8400,(02) 8671 9616,(02) 8671 9616,Government
6,71136,2021-08-27,MANILA DOCTORS HOSPITAL,ERMITA,9,9,18,14,15,38,6,9,(02) 8558 0888,None,(02) 8524 7376,Private
7,329387,2021-08-27,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,7,4,5,0,3,12,5,8,(02) 8716 1853,None,(02) 8716 1853,Private
8,59974,2021-08-27,DE LOS SANTOS MEDICAL CENTER,QUEZON CITY,6,12,5,3,2,16,2,4,(02) 8723 0041,(02) 8893 5762,None,Private
9,241594,2021-08-27,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,6,11,7,0,9,44,8,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government


In [27]:
show_filtered(df_cap, days_ago=2)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,10568,2021-08-28,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,80,0,30,69,60,0,33,311,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
1,64209,2021-08-28,JOSE R. REYES MEMORIAL MEDICAL CENTER,SANTA CRUZ,79,54,13,0,12,68,6,45,(02) 8711 9491,(02) 8711 9498,(02) 8732 1077,Government
2,501532,2021-08-28,EAST AVENUE MEDICAL CENTER,QUEZON CITY,27,0,23,0,108,229,18,0,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
3,39958,2021-08-28,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,14,0,2,0,34,117,2,0,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
4,71137,2021-08-28,MANILA DOCTORS HOSPITAL,ERMITA,9,5,17,14,15,42,7,9,(02) 8558 0888,None,(02) 8524 7376,Private
5,382616,2021-08-28,MAKATI MEDICAL CENTER,CITY OF MAKATI,9,0,12,28,20,87,8,37,(02) 8888 8999,(02) 8840 0737,(02) 8892 5593,Private
6,461877,2021-08-28,RIZAL MEDICAL CENTER,CITY OF PASIG,7,0,12,37,23,0,10,66,(02) 8865 8400,(02) 8671 9616,(02) 8671 9616,Government
7,59975,2021-08-28,DE LOS SANTOS MEDICAL CENTER,QUEZON CITY,6,12,5,5,2,16,2,2,(02) 8723 0041,(02) 8893 5762,None,Private
8,241595,2021-08-28,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,5,10,6,0,10,45,9,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
9,379654,2021-08-28,THE MEDICAL CITY,CITY OF PASIG,5,17,1,0,17,60,20,0,(02) 8988 1000,(02) 8988 7000,(02) 8635 3325,Private


In [28]:
show_filtered(df_cap, days_ago=1)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,64210,2021-08-29,JOSE R. REYES MEMORIAL MEDICAL CENTER,SANTA CRUZ,79,55,13,0,12,67,6,45,(02) 8711 9491,(02) 8711 9498,(02) 8732 1077,Government
1,10569,2021-08-29,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,77,0,33,77,63,0,30,303,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
2,501533,2021-08-29,EAST AVENUE MEDICAL CENTER,QUEZON CITY,63,21,23,0,83,170,18,0,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
3,39959,2021-08-29,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,19,0,4,0,29,146,0,0,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
4,382617,2021-08-29,MAKATI MEDICAL CENTER,CITY OF MAKATI,10,0,10,27,19,87,10,38,(02) 8888 8999,(02) 8840 0737,(02) 8892 5593,Private
5,71138,2021-08-29,MANILA DOCTORS HOSPITAL,ERMITA,9,5,17,14,15,42,7,9,(02) 8558 0888,None,(02) 8524 7376,Private
6,461878,2021-08-29,RIZAL MEDICAL CENTER,CITY OF PASIG,8,0,13,66,22,0,9,37,(02) 8865 8400,(02) 8671 9616,(02) 8671 9616,Government
7,21976,2021-08-29,NATIONAL CENTER FOR MENTAL HEALTH,CITY OF MANDALUYONG,8,504,2,407,11,46,6,284,(02) 8531 9001,None,(02) 8531 8682,Government
8,241596,2021-08-29,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,7,21,6,0,8,44,9,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
9,59976,2021-08-29,DE LOS SANTOS MEDICAL CENTER,QUEZON CITY,6,13,4,3,2,15,3,4,(02) 8723 0041,(02) 8893 5762,None,Private


In [29]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['II'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,420631,2021-08-29,"PLTC, INC. LUIS A. TIAM MEDICAL CENTER",SOLANO,1,0,1,14,2,7,2,4,+63 788080042,+63 788053682,None,Private


In [30]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['IV-A'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,388884,2021-08-29,BATANGAS MEDICAL CENTER,BATANGAS CITY (CAPITAL),13,28,9,2,36,66,21,11,+63 437408307,None,+63 437230165,Government
1,182258,2021-08-29,QUEZON MEDICAL CENTER,LUCENA CITY (CAPITAL),8,36,8,16,7,0,2,49,+63 427104023,None,None,Government
2,88447,2021-08-29,GENERAL EMILIO AGUINALDO MEMORIAL HOSPITAL,TRECE MARTIRES CITY (CAPITAL),4,12,10,16,3,7,0,14,+63 464190063,None,None,Government
3,104947,2021-08-29,"ST. FRANCES CABRINI MEDICAL CENTER, INC.",SANTO TOMAS,3,0,2,0,3,8,2,16,+63 487784811,None,+63 487784819,Private
4,100723,2021-08-29,CALAMBA DOCTORS? HOSPITAL,CITY OF CALAMBA,2,3,3,0,2,27,2,10,+63 495457371,None,+63 49 5452527,Private
5,312058,2021-08-29,MEDICAL CENTER IMUS,IMUS CITY,2,1,3,2,2,28,0,4,+63 464722220,None,+63 464723031,Private
6,429743,2021-08-29,"GLOBAL MEDICAL CENTER OF LAGUNA, INC.",CABUYAO CITY,2,7,3,0,1,12,0,0,+63 495445812,+63 495445921,None,Private
7,82121,2021-08-29,"METRO ANTIPOLO HOSPITAL AND MEDICAL CENTER, INC.",CITY OF ANTIPOLO (CAPITAL),1,1,1,1,3,7,1,4,None,None,None,Private
8,347567,2021-08-29,METRO LEMERY MEDICAL CENTER,LEMERY,1,0,1,0,2,12,2,0,+63 434090480,None,+63 437400245,Private
9,409195,2021-08-29,"LOS BA?OS DOCTORS HOSPITAL AND MEDICAL CENTER,...",LOS BAÑOS,1,12,2,0,4,9,2,0,+63 495360100,None,+63 495361225,Private


In [31]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['III'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,185639,2021-08-29,BATAAN GENERAL HOSPITAL AND MEDICAL CENTER,CITY OF BALANGA (CAPITAL),16,6,16,0,40,71,19,131,+63 472372269,None,None,Government
1,441151,2021-08-29,PRI MEDICAL CENTER,ANGELES CITY,7,1,3,0,23,7,1,0,None,None,None,Private
2,521846,2021-08-29,JOSE B. LINGAD MEMORIAL REGIONAL HOSPITAL,CITY OF SAN FERNANDO (CAPITAL),7,0,11,4,178,100,44,192,+63 459613544,None,None,Government
3,48824,2021-08-29,DR. PAULINO J. GARCIA MEMORIAL RESEARCH & MEDI...,CABANATUAN CITY,5,23,2,9,28,79,24,56,+63 444638286,+63 444639937,None,Government
4,96068,2021-08-29,BULACAN MEDICAL CENTER,CITY OF MALOLOS (CAPITAL),4,8,42,21,24,10,16,146,+63 447910630,None,None,Government
5,212899,2021-08-29,RAMOS GENERAL HOSPITAL,CITY OF TARLAC (CAPITAL),2,7,2,0,1,8,1,0,+63 459827074,None,None,Private
6,457644,2021-08-29,PRES. RAMON MAGSAYSAY MEMORIAL HOSPITAL,IBA (CAPITAL),2,0,1,29,8,28,4,141,+63 478117212,None,None,Government
7,517615,2021-08-29,ALLIED CARE EXPERTS (ACE) MEDICAL CENTER ?BALI...,BALIUAG,2,7,1,0,3,23,2,0,+63 448161000,None,+63 448161000,Private
8,112988,2021-08-29,TARLAC PROVINCIAL HOSPITAL,CITY OF TARLAC (CAPITAL),1,0,4,0,16,48,4,189,+63 459821306,None,None,Government
9,279262,2021-08-29,JAMES L. GORDON MEMORIAL HOSPITAL,OLONGAPO CITY,1,16,6,0,6,38,6,20,+63 476021229,+63 476021230,+63 476021228,Government


In [32]:
roi = reg_lookup['NCR']
#roi = reg_lookup['II']
#roi = reg_lookup['IV-A']
filter = (df_cap.cap_reportdate == day_ago_from(1, data_date)) & (df_cap.region == roi)
occupied = df_cap[filter]['icu_o'].sum()
vacant = df_cap[filter]['icu_v'].sum()
pct_used = (occupied / (vacant + occupied)) * 100

In [33]:
def dump_daily(df_cap, days_ago):
  filter = (df_cap.cap_reportdate == day_ago_from(days_ago, data_date)) 
  df_vacant = df_cap[filter][['cfname', 'region', 'province', 'city_mun', 
                              'icu_v', 'isolbed_v', 'mechvent_v', 'beds_ward_v', 
                              'icu_o', 'isolbed_o', 'mechvent_o', 'beds_ward_o', 
                              'Landline Number', 'Landline Number 2', 'Fax Number', 'Ownership Major Classification']]
  df_vacant.rename({'Landline Number': 'landline', 'Landline Number 2': 'landline2', 'Fax Number': 'fax', 'Ownership Major Classification':'ownership'}, axis=1, inplace=True)
  df_vacant['landline'] = df_vacant['landline'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['landline2'] = df_vacant['landline2'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['fax'] = df_vacant['fax'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant.sort_values(by='icu_v', ascending=False, inplace=True)
  df_vacant.reset_index(inplace=True, drop=True)
  return df_vacant

In [34]:
#https://stackoverflow.com/questions/29463274/simulate-autofit-column-in-xslxwriter
def get_col_widths(dataframe):
    # First we find the maximum length of the index column   
    idx_max = max([len(str(s)) for s in dataframe.index.values] + [len(str(dataframe.index.name))])
    # Then, we concatenate this to the max of the lengths of column name and its values for each column, left to right
    return [idx_max] + [max([len(str(s)) for s in dataframe[col].values] + [len(col)]) for col in dataframe.columns]

In [35]:
#writer = pd.ExcelWriter('hospitals_DOHDataDrop_' + data_date.strftime('%Y%m%d') + '.xlsx', engine='xlsxwriter')
writer = pd.ExcelWriter('hospitals_DOHDataDrop.xlsx', engine='xlsxwriter')
workbook  = writer.book
format_wrap = workbook.add_format({'text_wrap': True})

In [36]:
readme = 'README'
notes = []
notes.append('Data source: DOH Data Drop ' + data_date.strftime('%Y-%m-%d'))
notes.append('Data source link is in the README PDF @ ' + 'https://bit.ly/DataDropPH')
notes.append('Info here is dated -- call the ONE HOSPITAL COMMAND CENTER for current availability. Reach them through the ff. numbers: (02) 8865-0500, 0915-777-7777 and 0919-977-3333')
notes.append('cap_reportdate = date of report for icu, isolation rooms, beds in ward, ventilators')
notes.append('IMPORTANT: Please check https://nhfr.doh.gov.ph/rfacilities2list.php for latest hospital info')
notes.append('Default sorting is by number of vacant ICU beds - largest to smallest.')
notes.append('This is not an official document. It has not been validated - there could be errors.')
notes.append('Refer to ncovtracker.doh.gov.ph for official announcements')
notes.append('Very important: Please contact the ONE HOSPITAL COMMAND CENTER if you are looking for a hospital room / healthcare referral.')
notes.append('OHCC contact: (02) 8865-0500, 0915-777-7777 and 0919-977-3333')
notes.append('DOH COVID-19 toll-free hotline number: 1555')
notes.append('Alternatively, contact your LGU - https://www.rappler.com/nation/list-covid-19-hotline-numbers-one-hospital-command-metro-manila')
notes.append('Source code at https://github.com/maytlim/doh_data')
df_readme = pd.DataFrame(notes)
df_readme.to_excel(writer, readme)

In [37]:
for days_ago in range(1, 4):
  reportdate = day_ago_from(days_ago, data_date)
  df_dumpdaily = dump_daily(df_cap, days_ago)
  df_dumpdaily.to_excel(writer, reportdate)
  worksheet = writer.sheets[reportdate]
  worksheet.set_row(0, None, cell_format=format_wrap)
  for i, width in enumerate(get_col_widths(df_dumpdaily)):
    worksheet.set_column(i, i, min(20, max(width + 4, 5)))
  (max_row, max_col) = df_dumpdaily.shape
  # Set the autofilter
  worksheet.autofilter(0, 1, max_row, max_col)

In [38]:
writer.close()